## Przygotowanie datasetu

Ten segment kodu służy do przygotowania datasetu. Wczytuje pliki JSON zawierające informacje o podziale na zbiory treningowe, walidacyjne i testowe oraz duży plik z danymi dialogowymi. Kod jest zoptymalizowany do pracy z dużymi plikami JSON, zapewniając efektywne zarządzanie pamięcią.

### Szczegółowy opis:

1. **Definicja ścieżek do plików**:
   - `SPLIT_FILE_PATH`: Ścieżka do pliku zawierającego podział na zbiory treningowe, walidacyjne i testowe.
   - `NEWSDIALOG_FILE_PATH`: Ścieżka do dużego pliku JSON z dialogami.

2. **Wczytanie danych podziału**:
   - Dane z pliku `train_val_test_split.json` są wczytywane do zmiennej `split_data` przy użyciu biblioteki `json`.

3. **Funkcja do wczytywania dużego pliku JSON**:
   - `load_large_json(file_path)` to funkcja generatora, która wczytuje plik JSON linia po linii, umożliwiając przetwarzanie dużych plików bez obciążania pamięci.

4. **Potwierdzenie załadowania plików**:
   - Po wczytaniu danych wyświetla komunikat: „Files have been successfully loaded.”

In [2]:
import json
import os
import random

# File paths
SPLIT_FILE_PATH = "/Users/kamiljaworski/Projects/NLP_pro/datasets/MediaSum/data/train_val_test_split.json"
NEWSDIALOG_FILE_PATH = "/Users/kamiljaworski/Projects/NLP_pro/datasets/MediaSum/data/news_dialogue.json"

# Load split data
with open(SPLIT_FILE_PATH, "r") as split_file:
    split_data = json.load(split_file)

# Function to load a large JSON file line by line
def load_large_json(file_path):
    """
    Generator function to load a large JSON file line by line.
    
    Args:
        file_path (str): Path to the JSON file.
        
    Yields:
        dict: Parsed JSON object for each line in the file.
    """
    with open(file_path, "r") as file:
        for line in file:
            yield json.loads(line)

print("Files have been successfully loaded.")

Files have been successfully loaded.


## Pobieranie próbek z danych

Ten fragment kodu służy do pobierania określonej liczby próbek z dostępnych danych podziału na zbiory `train`, `val` i `test`. Przydatny jest w przypadkach, gdy potrzebujesz losowego podzbioru danych do eksperymentów lub testów.

### Szczegółowy opis:

1. **Funkcja `sample_train_ids`**:
   - Pobiera losową próbkę ID z połączonych zbiorów `train`, `val` i `test`.
   - Argumenty:
     - `split_data`: Słownik zawierający dane podziału.
     - `sample_size`: Liczba próbek do wybrania.
   - Funkcja drukuje liczbę unikalnych ID oraz podnosi błąd, jeśli żądana próba przekracza dostępne dane.

2. **Łączenie danych**:
   - Wszystkie ID ze zbiorów `train`, `val` i `test` są łączone w jedną listę.

3. **Próba losowa**:
   - Funkcja `random.sample` wybiera określoną liczbę ID w sposób losowy, bez powtórzeń.

4. **Zmienne globalne**:
   - `accumulated_ids`: Liczba wszystkich ID w danych.
   - `SAMPLE_SIZE`: Określa wielkość próby (tutaj równa liczbie wszystkich ID).

5. **Wynik**:
   - Wyświetlana jest liczba wybranych próbek.

In [3]:
def sample_train_ids(split_data, sample_size):
    """
    Selects a specified number of samples from the `train`, `val`, and `test` datasets.

    Args:
        split_data (dict): Dictionary containing split data.
        sample_size (int): Number of samples to select.

    Returns:
        list: List of selected IDs.
    """
    # Combine IDs from train, validation, and test splits
    train_ids = split_data["train"] + split_data["val"] + split_data["test"]
    
    # Print the number of unique IDs
    print(f"Number of unique IDs: {len(train_ids)}")
    
    # Raise an error if the sample size exceeds the total number of IDs
    if sample_size > len(train_ids):
        raise ValueError(
            f"Sample size ({sample_size}) exceeds the number of available data points ({len(train_ids)})."
        )
    
    # Randomly sample the specified number of IDs
    return random.sample(train_ids, sample_size)

# Calculate the total number of IDs across all splits
accumulated_ids = len(split_data["train"] + split_data["val"] + split_data["test"])

# Define the sample size as the total number of IDs
SAMPLE_SIZE = accumulated_ids

# Select a sample of IDs
sampled_ids = sample_train_ids(split_data, SAMPLE_SIZE)

# Print the number of selected samples
print(f"Selected {len(sampled_ids)} samples.")

Number of unique IDs: 463596
Selected 463596 samples.


## Wyodrębnianie danych na podstawie ID

Ten fragment kodu wyodrębnia artykuły i ich streszczenia na podstawie podanych ID z dużego pliku JSON. Wyodrębnione dane są następnie zapisywane do nowego pliku JSON.

### Szczegółowy opis:

1. **Funkcja `extract_data_by_ids`**:
   - Wyodrębnia artykuły zawierające dane `id`, `text`, oraz `summary` z pliku JSON.
   - Argumenty:
     - `news_file_path`: Ścieżka do pliku `NewsDialog.json`.
     - `ids_to_extract`: Lista ID, które mają zostać wyodrębnione.
   - Zwraca listę słowników z polami:
     - `id`: ID artykułu.
     - `text`: Połączony tekst artykułu.
     - `summary`: Streszczenie artykułu.

2. **Szybkie wyszukiwanie ID**:
   - Wykorzystano `set` dla szybszego sprawdzania obecności ID.

3. **Warunki zakończenia**:
   - Pętla zatrzymuje się po przetworzeniu wszystkich ID z listy, co przyspiesza działanie kodu.

4. **Zapis danych**:
   - Wyodrębnione dane są zapisywane do pliku `extracted_samples.json` w formacie JSON z wcięciami dla czytelności.

### Wynik:
- Kod wyświetla liczbę wyodrębnionych artykułów oraz ścieżkę do zapisanego pliku JSON.

In [4]:
import json

def extract_data_by_ids(news_file_path, ids_to_extract):
    """
    Extracts articles and summaries based on provided IDs.

    Args:
        news_file_path (str): Path to the NewsDialog.json file.
        ids_to_extract (list): List of IDs to extract.

    Returns:
        list: A list of articles containing ID, text, and summary.
    """
    extracted_data = []
    ids_set = set(ids_to_extract)  # Use a set for faster lookups

    # Open and load the JSON file
    with open(news_file_path, "r") as file:
        news_data = json.load(file)  # Load the entire JSON file (list)

        for article in news_data:
            # Check if "id" exists and matches one of the selected IDs
            if "id" in article and article["id"] in ids_set:
                extracted_data.append({
                    "id": article["id"],
                    "text": " ".join(article.get("utt", [])),  # Join sentences
                    "summary": article.get("summary", "")  # Get the summary
                })
            # Stop when all IDs have been processed
            if len(extracted_data) == len(ids_to_extract):
                break  

    return extracted_data


# Extract data
extracted_samples = extract_data_by_ids(NEWSDIALOG_FILE_PATH, sampled_ids)
print(f"Extracted {len(extracted_samples)} articles.")

# Save extracted data to a JSON file
output_file = "./datasets/extracted_samples.json"
with open(output_file, "w") as file:
    json.dump(extracted_samples, file, indent=4)
print(f"Extracted data saved to file: {output_file}")

Extracted 463596 articles.
Extracted data saved to file: ./datasets/extracted_samples.json


## Wyświetlanie losowych przykładów

Ten fragment kodu losowo wybiera dwa przykłady z wyodrębnionych artykułów i wyświetla ich zawartość, w tym ID, tekst oraz streszczenie.

### Szczegółowy opis:

1. **Losowy wybór przykładów**:
   - Używana jest funkcja `random.sample`, aby wybrać dwa losowe przykłady z listy `extracted_samples`.

2. **Wyświetlanie wyników**:
   - Dla każdego przykładu drukowane są:
     - `Article ID`: Unikalny identyfikator artykułu.
     - `Text`: Tekst artykułu (łączony z poszczególnych zdań).
     - `Summary`: Streszczenie artykułu.
   - Przykłady są oddzielone linią składającą się ze 100 znaków `-`, aby poprawić czytelność.

### Wynik:
- Po uruchomieniu kodu zobaczysz dwa losowe artykuły wybrane z wyodrębnionych danych, z pełnym tekstem i streszczeniem.

### Uwagi:
- Kod jest prosty i efektywny, pozwalając szybko zweryfikować zawartość wyodrębnionych danych.
- Może być łatwo zmodyfikowany, aby wybrać inną liczbę przykładów, zmieniając wartość w funkcji `random.sample`.

In [ ]:
# Wybierz dwa losowe przykłady
random_samples = random.sample(extracted_samples, 2)

# Wyświetl dwa losowe przykłady
for i, sample in enumerate(random_samples):  
    print(f"Article ID: {sample['id']}")
    print(f"Text:\n{sample['text']}\n")
    print(f"Summary:\n{sample['summary']}\n{'-'*100}")

Article ID: CNN-407734
Text:
As the total number of coronavirus cases in the U.S. nears 5 million, we're now learning just how quickly the virus is spreading inside the U.S. federal prison system. CNN's Drew Griffin has our report. The Seagoville Federal Correctional Institution outside Dallas is a petri dish of coronavirus infection. I lost my smell. I lived in the restroom for like 12 days. In the 15-minute phone call allowed from the inside, inmate George Reagan explained how coronavirus swept through the facility in just a month. Everybody has it now. Seagoville has significantly more COVID-19 cases than any other federal prison in the U.S. More than 1,300 of the roughly 1,750 inmates there tested positive. That's 75 percent. George Reagan's wife, Tabitha, that says no visitors have been allowed at the prison for months, so she believes workers must have infected the inmates. This was 100 percent their fault. COVID was brought in by their people. The FCI Seagoville staff was not pr

## Liczenie tokenów w tekście

Ten fragment kodu wykorzystuje tokenizer z biblioteki `transformers`, aby policzyć liczbę tokenów w podanym tekście. Jest to przydatne w przetwarzaniu języka naturalnego (NLP), szczególnie przy pracy z modelami opartymi na transformerach, takich jak BART.

### Szczegółowy opis:

1. **Funkcja `count_tokens`**:
   - Liczy liczbę tokenów w podanym tekście przy użyciu tokenizera `facebook/bart-base`.
   - Argumenty:
     - `text` (str): Tekst wejściowy, który ma zostać tokenizowany.
   - Zwraca liczbę tokenów w postaci liczby całkowitej (`int`).

2. **Użycie tokenizera BART**:
   - Tokenizer `BartTokenizer` jest inicjalizowany z pretrenowanego modelu `facebook/bart-base`.

3. **Przykład użycia**:
   - Dla tekstu `""" example text... """` liczba tokenów jest obliczana i wyświetlana w konsoli.

### Wynik:
- Kod wyświetla liczbę tokenów w podanym tekście, co pomaga w analizie i przygotowaniu danych do przetwarzania przez modele NLP.

### Uwagi:
- Kod wykorzystuje bibliotekę `transformers` firmy Hugging Face, co oznacza, że należy ją zainstalować przed uruchomieniem kodu (`pip install transformers`).
- Funkcja jest elastyczna i może być używana do analizy dowolnego tekstu.

In [6]:
from transformers import BartTokenizer

def count_tokens(text):
    """
    Counts the number of tokens in the given text.

    Args:
        text (str): Input text.

    Returns:
        int: Number of tokens in the text.
    """
    # Initialize the BART tokenizer
    tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
    
    # Tokenize the input text and return the token count
    tokens = tokenizer.encode(text)
    return len(tokens)

# Example usage
example_text = """The world will be finding out this morning just what city will host the 2008 Olympic Games. Members of the International Olympic Committee are choosing between five candidates. And if you're curious, in Las Vegas, odds makers have their money on Beijing. Well, CNN's Patrick Snell is covering the IOC meeting. He joins us now, live, from Moscow and that's where the announcement will be made in just a few hours. Hello there, what can you tell us? Linda, well welcome to the Russian capital. The very latest as the tension mounts here, the excitement we get all the more closer to who will win the right to stage the 2008 Summer Games. Let me bring you up to date what's happened so far today, Friday, in Moscow. We've had three presentations from the bid cities. The first of which was the Japanese bid from Osaka. Now, in this bid, the Japanese are claiming that they will provide the best sporting facilities and infrastructure in the world. And what's different -- what's innovative about this bid is that the facilities will be built across three uniquely designed manmade islands. So that's something that the Japanese are confident about. They're doubly more confident because, of course, they'll be staging next summer's football World Cup along with South Korea. So they're very, very confident that this is the bid that should catch the public imagination and hopefully will try and pull in, from their point of view, these 122 IOC votes that are up for grabs. Now what else do we have today? Well, we've had the bid from the French capital Paris. Yes, Oui Paris 2008. The French believe the Olympic Games should come to their city. Remember, they staged the 1998 football World Cup and won many plaudits for doing so, a wonderfully spectacular performance. They say their infrastructure is in place. They've got the Parisian Stadium, the Stade de France already built, infrastructure ready to go. And because they're promising games like Patrick Snell, thank you very much, from Moscow. TO ORDER A VIDEO OF THIS TRANSCRIPT, PLEASE CALL 800-CNN-NEWS OR USE OUR SECURE ONLINE ORDER FORM LOCATED AT www.fdch.com
"""
token_count = count_tokens(example_text)
print(f"Number of tokens: {token_count}")

Number of tokens: 452


## Filtrowanie i podział artykułów

Ten fragment kodu wykonuje filtrowanie artykułów na podstawie długości tekstu i streszczenia (mierzonej liczbą tokenów), a następnie dzieli dane na zbiory treningowe, walidacyjne i testowe. Wyniki są zapisywane do plików JSON.

### Szczegółowy opis:

1. **Filtrowanie artykułów (`filter_articles_by_token_length`)**:
   - Artykuły są filtrowane na podstawie dwóch kryteriów:
     - Maksymalna liczba tokenów w tekście (`max_tokens`).
     - Minimalna liczba tokenów w streszczeniu (`min_summary_tokens`).
   - Wykorzystywany jest tokenizer BART-a (`facebook/bart-base`), aby przeliczyć liczbę tokenów.

2. **Podział i zapis danych (`split_and_save_filtered_data`)**:
   - Dane są losowo tasowane i dzielone na trzy zbiory:
     - Treningowy (`train_pct` - 60%).
     - Walidacyjny (`val_pct` - 30%).
     - Testowy (`test_pct` - 10%).
   - Każdy zbiór jest zapisywany do osobnego pliku JSON w określonym folderze.

3. **Ścieżki plików**:
   - Plik wejściowy (`input_file`) zawiera dane wejściowe w formacie JSON.
   - Folder wyjściowy (`output_folder`) przechowuje wyniki podziału danych.

### Wynik:
- Po przetworzeniu danych wyświetlana jest liczba artykułów po filtrowaniu oraz liczba próbek w każdym zbiorze.
- Dane są zapisane do plików `train.json`, `val.json`, i `test.json`.

### Uwagi:
- Tokenizer `BartTokenizer` wymaga biblioteki Hugging Face Transformers (`pip install transformers`).
- Wartość `max_tokens` i `min_summary_tokens` można dostosować do specyficznych wymagań projektu.

In [7]:
from transformers import BartTokenizer
import json
import os
import random

# Initialize BART tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

def filter_articles_by_token_length(input_data, max_tokens=1024, min_summary_tokens=10):
    """
    Filters articles so that the text has a maximum number of tokens, 
    and the summary has a minimum number of tokens.

    Args:
        input_data (list): List of articles with "id", "text", and "summary" keys.
        max_tokens (int): Maximum number of tokens for the text.
        min_summary_tokens (int): Minimum number of tokens for the summary.

    Returns:
        list: List of articles meeting the conditions.
    """
    filtered_data = []
    for article in input_data:
        text_tokenized = tokenizer(article["text"], truncation=False)["input_ids"]
        summary_tokenized = tokenizer(article["summary"], truncation=False)["input_ids"]
        
        # Check conditions: text and summary lengths
        if len(text_tokenized) <= max_tokens and len(summary_tokenized) >= min_summary_tokens:
            filtered_data.append(article)
    return filtered_data

def split_and_save_filtered_data(input_file, output_folder, max_tokens=1024, min_summary_tokens=10, train_pct=0.6, val_pct=0.3, test_pct=0.1):
    """
    Filters articles, splits them into training, validation, and test sets, 
    and saves the results to JSON files.

    Args:
        input_file (str): Path to the JSON file with articles.
        output_folder (str): Folder where results will be saved.
        max_tokens (int): Maximum number of tokens for the text.
        min_summary_tokens (int): Minimum number of tokens for the summary.
        train_pct (float): Percentage of data for training.
        val_pct (float): Percentage of data for validation.
        test_pct (float): Percentage of data for testing.

    Returns:
        None
    """
    # Load data from file
    with open(input_file, "r") as file:
        data = json.load(file)

    # Filter articles
    print("Filtering articles based on token counts...")
    filtered_data = filter_articles_by_token_length(
        data, max_tokens=max_tokens, min_summary_tokens=min_summary_tokens
    )
    print(f"Number of articles after filtering: {len(filtered_data)}")

    # Shuffle and split data
    random.shuffle(filtered_data)
    total_samples = len(filtered_data)
    train_count = int(total_samples * train_pct)
    val_count = int(total_samples * val_pct)
    train_data = filtered_data[:train_count]
    val_data = filtered_data[train_count:train_count + val_count]
    test_data = filtered_data[train_count + val_count:]

    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Save splits to JSON files
    with open(os.path.join(output_folder, "train.json"), "w") as train_file:
        json.dump(train_data, train_file, indent=4, ensure_ascii=False)
    with open(os.path.join(output_folder, "val.json"), "w") as val_file:
        json.dump(val_data, val_file, indent=4, ensure_ascii=False)
    with open(os.path.join(output_folder, "test.json"), "w") as test_file:
        json.dump(test_data, test_file, indent=4, ensure_ascii=False)

    print(f"Data saved in folder: {output_folder}")
    print(f"Sample counts: Training = {len(train_data)}, Validation = {len(val_data)}, Testing = {len(test_data)}")

# File paths
input_file = "./datasets/extracted_samples.json"  # File with previously extracted data
output_folder = "./datasets/splits_filtered_with_summary"  # Folder for results

# Filter and save data
split_and_save_filtered_data(
    input_file, output_folder, max_tokens=1024, min_summary_tokens=40, train_pct=0.6, val_pct=0.3, test_pct=0.1
)

Filtering articles based on token counts...
Number of articles after filtering: 24692
Data saved in folder: ./datasets/splits_filtered_with_summary
Sample counts: Training = 14815, Validation = 7407, Testing = 2470
